# Library / Pakcages

In [1]:
# basic library
import os
import pandas as pd
import numpy as np
import sys

# pickle and .env
from dotenv import dotenv_values
import pickle

# Dataset

In [2]:
# parameter
link = {**dotenv_values('../.env.secret')}
path = link['RAW']

# filter missing values
missing_val = ['N/a', 'n/a', 'No', 'N\a', 'na', 'NA', np.nan]

# load data to df
market_df = pd.read_csv(path, sep = ',', na_values = missing_val, low_memory = False)
market_df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [3]:
# Drop unnecessary column
market_df = market_df.drop(columns = ['Unnamed: 0', 'user id'])

# Apply lowercase in data
market_df = market_df.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)
market_df.columns = market_df.columns.str.lower().str.replace(' ', '_')

# Show all columns in dataframe
pd.set_option('display.max_columns', None)
market_df.head()

,test_group,converted,total_ads,most_ads_day,most_ads_hour
0,ad,False,130,monday,20
1,ad,False,93,tuesday,22
2,ad,False,21,tuesday,18
3,ad,False,355,tuesday,10
4,ad,False,276,friday,14


# Data Cleaning

In [4]:
# Check Data type
market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   test_group     588101 non-null  object
 1   converted      588101 non-null  bool  
 2   total_ads      588101 non-null  int64 
 3   most_ads_day   588101 non-null  object
 4   most_ads_hour  588101 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 18.5+ MB


## Data Duplicate

In [5]:
# check general duplicate
print(f"Total General Duplicate: {market_df.duplicated().sum()}")

Total General Duplicate: 543623


## Null Checking

In [6]:
# check general null
market_df.isnull().sum()

test_group       0
converted        0
total_ads        0
most_ads_day     0
most_ads_hour    0
dtype: int64

## Category Check

In [7]:
check_cat = market_df.select_dtypes(include = ['bool', 'object'])
check_cat.head()

,test_group,converted,most_ads_day
0,ad,False,monday
1,ad,False,tuesday
2,ad,False,tuesday
3,ad,False,tuesday
4,ad,False,friday


## Attribute Check

In [8]:
market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   test_group     588101 non-null  object
 1   converted      588101 non-null  bool  
 2   total_ads      588101 non-null  int64 
 3   most_ads_day   588101 non-null  object
 4   most_ads_hour  588101 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 18.5+ MB


In [9]:
check_att = market_df[['test_group', 'converted', 'most_ads_day', 'most_ads_hour']]
check_att.nunique()

test_group        2
converted         2
most_ads_day      7
most_ads_hour    24
dtype: int64

In [10]:
for i in check_att.columns:
    print(f'{i.upper()} \t: {check_att[i].unique()}')
    print(check_att[i].value_counts())
    print(f'{'-' * 50} \n')

TEST_GROUP 	: ['ad' 'psa']
test_group
ad     564577
psa     23524
Name: count, dtype: int64
-------------------------------------------------- 

CONVERTED 	: [False  True]
converted
False    573258
True      14843
Name: count, dtype: int64
-------------------------------------------------- 

MOST_ADS_DAY 	: ['monday' 'tuesday' 'friday' 'saturday' 'wednesday' 'sunday' 'thursday']
most_ads_day
friday       92608
monday       87073
sunday       85391
thursday     82982
saturday     81660
wednesday    80908
tuesday      77479
Name: count, dtype: int64
-------------------------------------------------- 

MOST_ADS_HOUR 	: [20 22 18 10 14 13 19 11 12 16 21  3 23  4  8  0  2 15  1  6 17  7  9  5]
most_ads_hour
13    47655
12    47298
11    46210
14    45648
15    44683
10    38939
16    37567
17    34988
18    32323
9     31004
19    30352
21    29976
20    28923
22    26432
23    20166
8     17627
7      6405
0      5536
2      5333
1      4802
3      2679
6      2068
5       765
4       722


## Data Manipulation

In [11]:
market_df.head()

,test_group,converted,total_ads,most_ads_day,most_ads_hour
0,ad,False,130,monday,20
1,ad,False,93,tuesday,22
2,ad,False,21,tuesday,18
3,ad,False,355,tuesday,10
4,ad,False,276,friday,14


In [12]:
# list
weekday_list = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']

# weekly categorize
market_df.loc[market_df['most_ads_day'].isin(weekday_list), 'week_cat'] = 'weekday'
market_df.loc[~market_df['most_ads_day'].isin(weekday_list), 'week_cat'] = 'weekend'

market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   test_group     588101 non-null  object
 1   converted      588101 non-null  bool  
 2   total_ads      588101 non-null  int64 
 3   most_ads_day   588101 non-null  object
 4   most_ads_hour  588101 non-null  int64 
 5   week_cat       588101 non-null  object
dtypes: bool(1), int64(2), object(3)
memory usage: 23.0+ MB


## Data Filter

### Object

In [13]:
# string
string_columns = market_df.select_dtypes(include = ['object']).columns.tolist()

# Pengecualian kolom
excluded_columns = ['week_cat']

string_columns = [col for col in string_columns if col not in excluded_columns]
print(f'String Columns: \n{string_columns}')

String Columns: 
['test_group', 'most_ads_day']


In [14]:
def process_strings(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = df[column].str.lower()  # Ubah semua karakter ke huruf kecil
            df[column] = df[column].str.replace(r'\s{2,}', ' ', regex = True)  # Ganti spasi berlebih dengan satu spasi
    return df

market_df = process_strings(market_df, string_columns)
market_df.head()

,test_group,converted,total_ads,most_ads_day,most_ads_hour,week_cat
0,ad,False,130,monday,20,weekday
1,ad,False,93,tuesday,22,weekday
2,ad,False,21,tuesday,18,weekday
3,ad,False,355,tuesday,10,weekday
4,ad,False,276,friday,14,weekday


### Row

In [15]:
# before filter
print(f'Total rows: {market_df.shape[0]}')

# filter error data
market_df = market_df.loc[market_df['total_ads'] > 0]

# after filter
print(f'Total rows: {market_df.shape[0]}')

Total rows: 588101
Total rows: 588101


# Write Data

In [16]:
market_df.columns

Index(['test_group', 'converted', 'total_ads', 'most_ads_day', 'most_ads_hour',
       'week_cat'],
      dtype='object')

In [17]:
# Nama direktori
dir_name = '../datamart'

# Mengecek apakah direktori sudah ada
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
    print(f"Directory '{dir_name}' created.")

else:
    print(f"Directory '{dir_name}' already exists.")

Directory '../datamart' created.


In [18]:
# calculate dataframe size
total_size_bytes = sys.getsizeof(market_df)

# Converse bytes to MB
total_size_mb = total_size_bytes / 1048576
print(f"Total size of DataFrame: {total_size_mb:.2f} MB")

Total size of DataFrame: 105.52 MB


In [19]:
# parameter
share = {**dotenv_values('../.env.shared')} 

# save to pickle
with open(share['PREPARATION_DATA'], 'wb') as f:
    pickle.dump(market_df, f)

print('Data has been saved')

Data has been saved
